In [1]:
# %pip install gym_super_mario_bros==7.3.0 nes_py

In [2]:
import numpy as np
import pandas as pd

In [3]:
#import the game
import gym_super_mario_bros
#import the joypad wrapper
from nes_py.wrappers import JoypadSpace
#import the simplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [4]:
import gym

class CustomJoypadSpace(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def reset(self, **kwargs):
        return super().reset()

In [5]:
# %pip install torch torchvision torchaudio

In [6]:
# %pip install stable-baselines3[extra]



In [7]:
#import framestacker and grayscaler
from gym.wrappers import GrayScaleObservation
#import vectorization wrappers
from stable_baselines3.common.vec_env import VecFrameStack,DummyVecEnv
#import matplotlib to show framestacking
from matplotlib import pyplot as plt

In [8]:
# JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs)

#1.Create base environment
env=gym_super_mario_bros.make('SuperMarioBros-v0',apply_api_compatibility=True,render_mode='human')
#2.Simplify Controls
env=JoypadSpace(env,SIMPLE_MOVEMENT)
#3.Grayscale
env=GrayScaleObservation(env,keep_dim=True)
#4.Wrap inside Dummy env
env=CustomJoypadSpace(env)
env=DummyVecEnv([lambda:env])
#5.Stack the frames
env=VecFrameStack(env,4,channels_order='last')

c:\Users\vaibh\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
c:\Users\vaibh\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\envs\registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(
c:\Users\vaibh\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [9]:
state=env.reset()

In [10]:
state,reward,done,info,_=env.step([env.action_space.sample()],apply_api_compatibility=True,zrender_mode="human")
plt.imshow(state[0])

TypeError: VecEnv.step() got an unexpected keyword argument 'apply_api_compatibility'

In [19]:
env.close()

ValueError: env has already been closed.

Training RL Model

In [12]:
import os
# os is for path management
from stable_baselines3 import PPO
#ppo is for algos
from stable_baselines3.common.callbacks import BaseCallback
# base callback is for saving models



In [13]:
# training and saving model

class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [14]:
CHECKPOINT_DIR="./train/"
LOG_DIR="./logs/"

In [15]:
#Setup and saving callbacks
callback=TrainAndLoggingCallback(check_freq=10000,save_path=CHECKPOINT_DIR)

In [16]:
#creating Model
model=PPO('CnnPolicy',env,verbose=1,tensorboard_log=LOG_DIR,learning_rate=0.000001,n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [17]:
#training the model 
model.learn(total_timesteps=1000000,callback=callback)

OSError: exception: access violation reading 0x000000000003C208